In [1]:
import rasterio
import rasterio.mask
import rasterio.crs
import geopandas as gpd
import shapely

In [2]:
shapefile = gpd.read_file("C:/Users/Eric W/Downloads/DEM_Extent_RusticCanyon (1)/DEM_Extent_RusticCanyon.shp")
shapefile = shapefile.to_crs('EPSG:2229')

In [3]:
shp_geom = []
shp_coord = shapely.geometry.mapping(shapefile)["features"][0]["geometry"]
shp_geom.append(shp_coord)

In [4]:
src = rasterio.open("C:/Users/Eric W/Box/Projects/LA County/WRAMPS/3. Data/GIS/HRU_Remap/WMMS2_HRUraster_AllHRUs.tif", 'r+')
src.crs = rasterio.crs.CRS.from_string('EPSG:2229')

In [5]:
out_image, out_transform = rasterio.mask.mask(src, shp_geom, crop=True)
out_meta = src.meta

In [6]:
out_meta.update({"driver": "GTiff",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform})

with rasterio.open("C:/Users/Eric W/Box/Projects/City of LA/SCW Feasibility Studies 2020/3_Data/GIS/Shapefiles/HRU Clips/HRU_RusticCanyon_4_21_20_V2.tif", "w", **out_meta,compress='LZW',threads = 4) as dest:
    dest.write(out_image)